# Open rice

In this exercise we'll use folium to help visulize the open rice dataset. You may find [these](http://nbviewer.jupyter.org/github/python-visualization/folium/tree/master/examples/) reference notebooks helpfull

## Markers

In [1]:
import folium
import pandas as pd
import json

In [2]:
hkjson = json.load(open("data/hk_geo.json"))

In [3]:
df = pd.read_csv('data/geotagged-openrice.csv')

** 1. ** Use folium to draw a pop up marker for the location of 10 resturants.

In [12]:
l_latlon = df[['latitude','longitude']].values[:10]
start_loc = list(l_latlon[0])

[22.2852, 114.1478]

### JHO's Note
The map tiles matter! Read [this](https://python-graph-gallery.com/288-map-background-with-folium/)

And this [Kaggle Tutorial](https://www.kaggle.com/daveianhickey/how-to-folium-for-maps-heatmaps-time-analysis) is also very useful

In [29]:
# init map
map_openrice = folium.Map( location = start_loc, zoom_start= 15, tiles = 'Stamen Toner')

for i in l_latlon:
    folium.Marker( list(i), popup='London Bridge').add_to(map_openrice)
map_openrice

In [ ]:
# %load solutions/draw_markers.py

In [8]:
df[["latitude","longitude"]].values[:10]

array([[ 22.2852, 114.1478],
       [ 22.2852, 114.1478],
       [ 22.2835, 114.1511],
       [ 22.2835, 114.1511],
       [ 22.2835, 114.1511],
       [ 22.2853, 114.1477],
       [ 22.2853, 114.1477],
       [ 22.2879, 114.1518],
       [ 22.2879, 114.1518],
       [ 22.2867, 114.1479]])

## Choropleth 

Before we can create a choropleth map we need to calculate if a point falls within a district. We can use shapely to calculate if a point falls within a polygon.

In [50]:
from geojson import utils #used to get coordinates from geo json
from shapely.geometry import Point, Polygon
import numpy as np

In [51]:
polygons  = [ Polygon(utils.coords(each)) for each in hkjson["features"]] 
points = [ Point(p) for p  in df[["longitude","latitude"]].values]

In [52]:
def find_district(polygons,districts,point):
    
    assert len(districts) == len(polygons)
    
    for i,polygon in enumerate(polygons):
        if polygon.contains(point):
            return districts[i]
    return np.nan


In [53]:
find_district(polygons,districts,points[1])

'Central and Western'

In [54]:
#quite slow be patient
resturant_district = [find_district(polygons,districts,point) for point in points]

In [48]:
df["district"] = resturant_district

In [55]:
df.head()

,address,bookmarks,dislikes,food_type,likes,name,number_of_reviews,price_range,latitude,longitude,district
0,"Shop J-K., 200 Hollywood Road,",5838,6,Hong Kong Style,78,For Kee Restaurant 科記咖啡餐室,(133 Reviews),Below $50,22.2852,114.1478,Central and Western
1,"Shop J-K., 200 Hollywood Road,",5838,6,Tea Restaurant,78,For Kee Restaurant 科記咖啡餐室,(133 Reviews),Below $50,22.2852,114.1478,Central and Western
2,"G/F, 108 Hollywood Road,",3492,2,International,20,Blue · Butcher & Meat Specialist,(30 Reviews),$201-400,22.2835,114.1511,Central and Western
3,"G/F, 108 Hollywood Road,",2259,10,Western,35,Classified,(71 Reviews),$101-200,22.2835,114.1511,Central and Western
4,"G/F, 108 Hollywood Road,",3492,2,International,20,Blue · Butcher & Meat Specialist,(30 Reviews),$201-400,22.2835,114.1511,Central and Western


We could then get the counts for all of a paritcular food type in a district like so.

In [56]:
df[df["food_type"] == "International"].district.value_counts()

Yau Tsim Mong          218
Central and Western    120
Wan Chai                87
Eastern                 64
Kwun Tong               64
Sha Tin                 40
Sham Shui Po            37
Southern                35
Kowloon City            35
Tsuen Wan               30
Yuen Long               28
Sai Kung                26
Kwai Tsing              25
Wong Tai Sin            21
Tai Po                  18
North                   16
Tuen Mun                11
Islands                  9
Name: district, dtype: int64

** 1. ** Use district counts like the one above to create a choropleth map to visulize the resturant styles in Hong Kong. You can use these notebooks a a reference.


* [Choropleth](http://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/GeoJSON_and_choropleth.ipynb)
* [Time Slider choropleth](http://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/TimeSliderChoropleth.ipynb)

In [ ]:
# %load solutions/choropleth.py